In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.feature_extraction.text  import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss, confusion_matrix, roc_curve, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from scipy import sparse
import re

seed = 42

In [2]:
train = pd.read_csv('train.csv').fillna(' ')
test  = pd.read_csv('test.csv').fillna(' ')

In [3]:

train.shape

(95851, 8)

In [4]:
train_text = train['comment_text']
test_text  = test['comment_text']

In [5]:
all_text = pd.concat([train_text,test_text])

In [6]:
import nltk
nltk.download()


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
from nltk.corpus import stopwords
stemmer = SnowballStemmer('english')

In [7]:
def clean(single_comment):
    letters_only = re.sub("[^a-zA-Z]", " ", single_comment) 
    words = letters_only.lower().split() 
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops] 
    stemmed_words = [stemmer.stem(w) for w in meaningful_words]
    return( " ".join( stemmed_words ))
    

In [8]:
clean_train_comments = []
num_train = len(train_text)
for i in range( 0, num_train ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_train ))                                                                    
    clean_train_comments.append( clean( train_text[i] ))

Review 1000 of 95851

Review 2000 of 95851

Review 3000 of 95851

Review 4000 of 95851

Review 5000 of 95851

Review 6000 of 95851

Review 7000 of 95851

Review 8000 of 95851

Review 9000 of 95851

Review 10000 of 95851

Review 11000 of 95851

Review 12000 of 95851

Review 13000 of 95851

Review 14000 of 95851

Review 15000 of 95851

Review 16000 of 95851

Review 17000 of 95851

Review 18000 of 95851

Review 19000 of 95851

Review 20000 of 95851

Review 21000 of 95851

Review 22000 of 95851

Review 23000 of 95851

Review 24000 of 95851

Review 25000 of 95851

Review 26000 of 95851

Review 27000 of 95851

Review 28000 of 95851

Review 29000 of 95851

Review 30000 of 95851

Review 31000 of 95851

Review 32000 of 95851

Review 33000 of 95851

Review 34000 of 95851

Review 35000 of 95851

Review 36000 of 95851

Review 37000 of 95851

Review 38000 of 95851

Review 39000 of 95851

Review 40000 of 95851

Review 41000 of 95851

Review 42000 of 95851

Review 43000 of 95851

Review 44000 of 9585

In [14]:
train_text[7]

'Guy Fawkes \n\nim a resident in bridgwater and i go to the carnival every year, im wuite dedicated to the town and enjoy alot about it, However, i think you are wrong, although we are rebels i think that the carnvial is to celebrate Guy fawkes getting caught and therfor preventing it, why would you want to celebrate somebody theying to murder out majisty.'

In [15]:
clean_train_comments[7]

'guy fawk im resid bridgwat go carniv everi year im wuit dedic town enjoy alot howev think wrong although rebel think carnvial celebr guy fawk get caught therfor prevent would want celebr somebodi they murder majisti'

In [74]:
vect_word  = TfidfVectorizer(sublinear_tf = True, max_features = 15000,ngram_range=(1,1),stop_words=None,analyzer = 'word',tokenizer=None, preprocessor=None)
vect_word2  = TfidfVectorizer(sublinear_tf = True, max_features = 15000,ngram_range=(1,2),stop_words=None,analyzer = 'word',tokenizer=None, preprocessor=None)
vect_char  = TfidfVectorizer(sublinear_tf = True, max_features = 20000,stop_words=None,analyzer = 'char',tokenizer=None, preprocessor=None)


In [63]:
vect_word.fit(clean_train_comments)
train_word = vect_word.transform(clean_train_comments)
X = sparse.hstack([train_word])





/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [32]:
vect_word.fit(clean_train_comments)
train_word = vect_word.transform(clean_train_comments)
vect_char.fit(clean_train_comments)
train_char = vect_char.transform(clean_train_comments)
X = sparse.hstack([train_word, train_char])




/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [34]:
#word_freq = pd.DataFrame({'term': vect_word.get_feature_names(), 'occurrences':np.asarray(train_word.sum(axis=0)).ravel().tolist()})
#word_freq['frequency'] = word_freq['occurrences']/np.sum(word_freq['occurrences'])


In [75]:
vect_word.fit(clean_train_comments)
train_word = vect_word.transform(clean_train_comments)
vect_char.fit(clean_train_comments)
train_char = vect_char.transform(clean_train_comments)
vect_word2.fit(clean_train_comments)
train_word2 = vect_word2.transform(clean_train_comments)
X = sparse.hstack([train_word, train_word2, train_char])

/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [69]:
#X_train = sparse.hstack([train_word,train_vector1,train_vector_char])
#X_test  = sparse.hstack([test_vector,test_vector1,test_vector_char])
#X_train = sparse.hstack([train_vector,train_vector_char])
#X_test  = sparse.hstack([test_vector,test_vector_char])

#Y_train = train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
#X = sparse.hstack([train_word])


X = sparse.hstack([train_word, train_word2, train_char])


In [76]:
Y = train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size = 0.3, random_state =42  )

In [13]:
target_col = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [36]:
train_word.shape

(95851, 15000)

## Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
lr = LogisticRegression(solver='newton-cg')

In [ ]:
# lr.fit(X_train,Y_train)

In [43]:
# test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [85]:
submission = pd.DataFrame(test['id'])
for target in target_col:
    Y_col = Y_train[target]
    lr.fit(X_train,Y_col)
    submission[target] = lr.predict_proba(X_test)[:,1]

In [86]:
submission.to_csv('submission3.csv',index=False)

In [87]:
submission.shape

(153164, 7)

In [106]:
s = pd.read_csv('submission.csv')

In [73]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997609,0.171832,0.995936,0.034118,0.944175,0.306862
1,0000247867823ef7,0.006958,0.002642,0.004620,0.000504,0.001299,0.001176
2,00013b17ad220c46,0.022850,0.001113,0.012370,0.000469,0.006027,0.001558
3,00017563c3f7919a,0.006423,0.003306,0.003889,0.000724,0.004597,0.000922
4,00017695ad8997eb,0.032664,0.001663,0.003319,0.001973,0.012162,0.001672


In [77]:
sum = 0
for target in target_col:
    Y_train_col = Y_train[target]
    Y_test_col  = Y_test[target]
    lr.fit(X_train, Y_train_col)
    sum += roc_auc_score( Y_test_col,pd.DataFrame(lr.predict_proba(X_test)[:,1]))
    print(target)
    print(roc_auc_score( Y_test_col,pd.DataFrame(lr.predict_proba(X_test)[:,1])))
sum = sum/6

toxic
0.9710665846220728
severe_toxic
0.9829155107661988
obscene
0.9882546687321436
threat
0.9678713458452521
insult
0.9758868669054923
identity_hate
0.9717236438858596


In [78]:
sum

0.9762864367928366

# NB-SVM

In [160]:
vec = TfidfVectorizer(sublinear_tf = True, max_features = 60000,ngram_range=(1,2),stop_words=None,analyzer = 'word',tokenizer=None, preprocessor=None)


In [161]:
vec.fit(clean_train_comments)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=1,
        vocabulary=None)

In [162]:
train_term = vec.transform(clean_train_comments)

/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [163]:
def pr(y_i,y):
    p = x[y==y_i].sum(0)
    return (p+1)/((y == y_i).sum() + 1)

In [164]:
X_train, X_test, Y_train, Y_test= train_test_split(train_term, Y, test_size = 0.3, random_state =42  )

In [165]:
x = X_train.sign()
x_test  = X_test.sign()

In [166]:
def get_model(y):
    y = y.values
    r = np.log(pr(1,y)/pr(0,y))
    lr_model= LogisticRegressionCV(Cs=5)
    x_nb = x.multiply(r)
    return lr_model.fit(x_nb, y), r
    

In [167]:
preds = np.zeros((len(Y_test), len(target_col)))

In [168]:
for i,j in enumerate(target_col):
    print('fit',j)
    m, r = get_model(Y_train[j])
    preds[:,i] = m.predict_proba(x_test.multiply(r))[:,1]

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [169]:
preds

array([[1.49868270e-03, 2.98560278e-03, 3.46954205e-04, 1.58838835e-03,
        1.91344924e-04, 1.53232748e-05],
       [3.67928535e-03, 1.95964033e-03, 9.05174995e-06, 1.14310277e-03,
        9.23604447e-05, 1.53608682e-05],
       [1.36786132e-05, 3.79157553e-03, 1.91211975e-05, 1.26681032e-03,
        5.68340429e-06, 3.89031479e-05],
       ...,
       [1.00000000e+00, 7.30573220e-01, 1.00000000e+00, 1.19879833e-02,
        9.99499622e-01, 3.91713011e-04],
       [8.57500980e-03, 3.78909416e-03, 3.98026478e-03, 2.36292210e-03,
        3.72995971e-03, 3.46779160e-04],
       [2.54360538e-01, 5.51600934e-03, 2.88158626e-02, 2.80371151e-03,
        3.26459135e-03, 9.12040384e-05]])

In [170]:
sum = 0
for target in range(6):
    Y_test_col  = Y_test.iloc[:,target]
    sum += roc_auc_score( Y_test_col,pd.DataFrame(preds[:,target]))
    print(target)
    print(roc_auc_score( Y_test_col,pd.DataFrame(preds[:,target])))
sum = sum/6

0
0.9546119867039577
1
0.973901156658976
2
0.973257004080167
3
0.9681980976301774
4
0.9570237635440719
5
0.9469827123418879


In [171]:
sum

0.962329120159873

In [116]:
preds[:,0]

array([1.50303387e-02, 2.18453710e-02, 6.31311349e-05, ...,
       1.00000000e+00, 4.53097623e-03, 6.94762981e-03])